# 04: Automatic Blog Post Creator

> "Discover the power of Artificial Intelligence with OpenAI's powerful API to generate text and images in projects". Udemy course.

In [165]:
import os
import openai 

In [166]:
#os.environ["OPENAI_API_KEY"] = "sk-"
openai.api_key = os.getenv("OPENAI_API_KEY")

## Git Automatic Update

In [167]:
from git import Repo
import git
from pathlib import Path

In [168]:
pwd

'/home/tommaselli/Documents/GPT-Projects/04-Post_Creator'

In [169]:
PATH_BLOG_REPO = Path('/home/tommaselli/Documents/GPT-Projects/.git')
PATH_BLOG = PATH_BLOG_REPO.parent 
PATH_CONTENT =PATH_BLOG/'content'

In [170]:
PATH_CONTENT.mkdir(exist_ok=True,parents=True)

In [171]:
def update_blog(commit_message='updates blog'):
    # GitPython -- Repo Locatio 
    repo = Repo(PATH_BLOG_REPO)
    # git add .
    repo.git.add(all=True)
    # git commit -m "updates blog"
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

update in github (working):

In [172]:
random_text = 'orkepofmkmwẽw'

In [173]:
#with open(PATH_BLOG/'index.html', 'w') as f:
#    f.write(random_text)

#update_blog()

## Insert HTML with GPT

In [174]:
import shutil

In [175]:
def create_new_blog(title, content, cover_image): 
    cover_image = Path(cover_image)

    files = len(list(PATH_CONTENT.glob('*.html')))
    new_title = f'{files+1}.html'
    path_new_content = PATH_CONTENT/new_title

    shutil.copy(cover_iamge, PATH_CONTENT)
    if not os.path.exists(path_new_content):
        # WRITE A NEW HTML FILE
        with open(path_new_content, 'w') as f:
            f.write('<!DOCTYPE html>\n')
def create_new_blog(title, content, cover_image): 
    cover_image = Path(cover_image)

    files = len(list(PATH_CONTENT.glob('*.html')))
    new_title = f'{files+1}.html'
    path_new_content = PATH_CONTENT/new_title

    shutil.copy(cover_image, PATH_CONTENT)
    if not os.path.exists(path_new_content):
        # WRITE A NEW HTML FILE
        with open(path_new_content, 'w') as f:
            f.write('<!DOCTYPE html>\n')
            f.write('<html>\n')
            f.write('<head>\n')
            f.write('<title>' + title + '</title>\n')
            f.write('</head>\n')
            f.write('<body>\n')
            f.write('<header>\n')
            f.write('<h1>' + title + '</h1>\n')
            f.write('</header>\n')
            f.write('<img src="' + cover_image.name + '">\n')
            ### GPT completion

            f.write(content.replace('\n', '<br>\n'))
            f.write('</body>\n')
            f.write('</html>\n')
            print('Blog created')
            return path_new_content
            
    else: 
        raise FileExistsError('File already exists')

In [176]:
path_new_content = create_new_blog('Test Blog', 'This is a test blog', 'cover.png')

Blog created


In [177]:
from bs4 import BeautifulSoup as Soup

In [178]:
with open(PATH_BLOG/'index.html') as index:
    soup = Soup(index.read())

In [179]:
str(soup)

'<html>\n<head>\n<title>\n   Page\n  </title>\n</head>\n<body>\n</body>\n</html>\n'

In [180]:
# checking duplicate links 
def check_duplicate(path_new_content, links):
    urls = [str(link.get('href')) for link in links] # 1.html, 2.html, 3.html...
    content_path = str(Path(*path_new_content.parts[-2:])) # (rel_path)/1.html, check if 1.html is in urls
    return content_path in urls

In [181]:
def write_index(path_new_content):
    with open(PATH_BLOG/'index.html') as index:
        soup = Soup(index.read())

    links = soup.find_all('a')
    try: 
        last_link = links[-1]
    except: 
        last_link = soup.new_tag('a')
        soup.append(last_link)

    if check_duplicate(path_new_content, links):
        raise ValueError('Duplicate link found')

    link_new_blog = soup.new_tag('a', href=Path(*path_new_content.parts[-2:]))
    link_new_blog.string = path_new_content.name.split('.')[0]
    last_link.insert_after(link_new_blog)

    with open(PATH_BLOG/'index.html', 'w') as index:
        index.write(str(soup.prettify(formatter='html')))

In [182]:
write_index(path_new_content)
update_blog('Added new blog')

# Add GPT

In [183]:
def create_prompt(title):
    prompt = ''' 
    Biography: My name is Felipe and I am a Python professional and instructor for coding.

    Blog
    Title: {}
    tags: tech, python, coding, AI machine learning
    Summarry: I talk about what the future of AI could hold for python developers
    Full text: 
    '''.format(title)
    return prompt 

In [184]:
title = 'The future of Python and AI'
prompt = create_prompt(title)
prompt

' \n    Biography: My name is Felipe and I am a Python professional and instructor for coding.\n\n    Blog\n    Title: The future of Python and AI\n    tags: tech, python, coding, AI machine learning\n    Summarry: I talk about what the future of AI could hold for python developers\n    Full text: \n    '

In [185]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt},
    ],
    temperature=0.75,
    max_tokens=512,
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable